*please execute the cell below before starting the tutorial*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit
from aiida_fleur.workflows.scf import FleurScfWorkChain
from aiida_fleur.workflows.mae import FleurMaeWorkChain

# Workchains and high-throughput study

Workchains are key-turn solutions for a particular problem. They are essential part of AiiDA package. The main task of a workchain is to automatically perform a certain task keeping the history of all performed actions.
In this part of the tutorial you will explore and learn how to work with AiiDA-Fleur workchains.

Wokchains behave themselves similarly to a calculation: basically it can be treated as a black box that uses user's input and produces some output. For example, the SCF workchain is an engine that generates self-consistent charge denstity and other parameters for a given structure.

The other important this is that workchains can be used in a higer-level workchains.

## Workchain hierarchy 

That is the hierarchy of all implemented workchains in AiiDA-Fleur v1.0.0. Black arrows mean the use of a workchain by another one. For instance, the Structure Optimization WorkChain uses the SCF workchain inside it.
It is clealy seen that every higher-level workchain uses the SCF one. The SCF workchain itself uses FleurCalculation.

<img src="files/images/workchains.png" width="600">

In next section we will cover the SCF workchain.

## SCF WorkChain 

The SCF workchain is responsible for converging charge density for a given structure.

### Inputs of the SCF WorkChain

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|fleur | Code | Fleur code | yes |
|inpgen | Code | Inpgen code| no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |
|calc_parameters | Dict | FLAPW parameters, used by inpgen| no |
|fleurinp | FleurinpData | FLEUR input files| no |
|remote_data | RemoteData | Remote folder of the previous calculation| no |
|options | Dict | AiiDA options (computational resourses)| no |
|settings | Dict | special settings for Fleur calculation| no |

Only Fleur code is required but it does not mean that one can run the SCF workchain giving only Fleur code. You **must** to follow on of the supported input configuration templates:

* fleur + fleurinp
* fleur + fleurinp + remote_data
* fleur + remote_data
* fleur + inpgen + structure

In other case the workchain will thow an error or more dangerously, perform an unexpected task.

In this tutorial we will cover only the last input configuration: we need to specify Fleur and inpgen codes and a structure. We also want to control workchain behaviour and do not use defaults for computational resources.

Firs of all, let us setup input parameters for the workchain:

In [ ]:
wf_para = Dict(dict={'fleur_runmax' : 3,           # maximal number of Fleur submissions
                     'density_converge' : 0.001,   # density convergence criterion
                     'mode' : 'density',           # we choose density to converge
                     'itmax_per_run' : 30,         # number of iterations in each Fleur submission
                     'serial' : False})            # we use mpi version of Fleur

Next, define computational resourses. Since we submit Fleur calculation to the local virtual machine, it is enough to specify total number of mpi tasks:

In [ ]:
options = Dict(dict={'resources' : {"num_mpiprocs_per_machine" : 2}})

We have already created StructureData node in the tutorial number 1. Let us use the stored value:

structure = load_node(xxx)

Parameters for inpgen code. For now we set k-mesh only:

In [ ]:
calc_parameters = Dict(dict={
    'kpt': {
        'div1': 12,
        'div2' : 10,
        'div3' : 1
        }})

Finally, we need to load Fleur and inpgen nodes:

In [ ]:
fleur_code = load_node(xxx)
inpgen_code = load_node(xxx)

Submit the SCF workchain:

In [ ]:
workchain_pk = submit(FleurScfWorkChain,
                      fleur=fleur_code,
                      inpgen=inpgen_code,
                      calc_parameters=calc_parameters,
                      structure=structure,
                      wf_parameters=wf_para)
print('Submitted SCF workchain pk={}'.format(workchain_pk))

## Higher-level workchains

And now - finally - we will see the real power of the workchains. Remember, how you struggled with tutorials on Monday and Tuesday? The stuggle is finished. Let us repeat all of these calculations in an hour.

### Magnetic anisotropy workchain 

A table of possible inputs looks the same as for SCF:

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|fleur | Code | Fleur code | yes |
|inpgen | Code | Inpgen code| no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |
|calc_parameters | Dict | FLAPW parameters, used by inpgen| no |
|fleurinp | FleurinpData | FLEUR input files| no |
|remote_data | RemoteData | Remote folder to find cdn1| no |
|options | Dict | AiiDA options (computational resourses)| no |

Again we see a lot of optional inputs - but you **must** follow one of the supported imput configurations. They are similar to the SCF workchain and I will not cover all of them here, let us just use the `fleur + inpgen + structure` mode. Let us also specify workchain parameters and computational resourses.

In [ ]:
options = Dict(dict={'resources' : {"num_mpiprocs_per_machine" : 2}})

In [ ]:
wf_para = Dict(dict={'sqa_ref': [0.7, 0.7],                    # theta and phi for reference calculation
                     'use_soc_ref': False,                     # True if switch SOC on for reference
                     'sqas_theta': [0.0, 1.57079, 1.57079],    # a list of theta values to calculate
                     'sqas_phi': [0.0, 0.0, 1.57079],          # a list of phi values to calculate
                     'fleur_runmax': 10,                       # passed to SCF workchain
                     'density_converged': 0.02,                # passed to SCF workchain
                     'serial': False,                          # passed to SCF workchain
                     'itmax_per_run': 30                       # passed to SCF workchain
                    })

In [ ]:
calc_parameters = Dict(dict={
    'kpt': {
        'div1': 12,
        'div2' : 10,
        'div3' : 1
        }})

In [ ]:
fleur_code = load_node(xxx)
inpgen_code = load_node(xxx)

In this tutorial we want to be even more productive - let us define not a single structure but three of them! Import Fe, Co and Ni structures:

In [ ]:
fe_structure = load_node(xxx)
co_structure = load_node(xxx)
ni_structure = load_node(xxx)

input_structures = [fe_structure, co_structure, ni_structure]

And calculate magnetic anisotropy energy for all of them:

In [ ]:
for structure in input_structures:
    workchain_pk = submit(FleurMaeWorkChain,
                          fleur=fleur_code,
                          inpgen=inpgen_code,
                          calc_parameters=calc_parameters,
                          structure=structure,
                          wf_parameters=wf_para)
    print('Submitted Mae workchain pk={} for {} structure'.format(workchain_pk, structure.get_formula()))

Now you let us proceed to the final task - Spin spiral dispersion workchain.

### Spin-spiral dispersion workchain

Spin spiral dispersion workchain has the same input nodes as MAE workchain. This time we will cover another input configuration: `fleur + fleurinp`. We will use FleurinpData objects generated in section 2. To use them, type:

In [ ]:
inpgen_calc_Fe = load_node(xxx)
inpgen_calc_Co = load_node(xxx)
inpgen_calc_Ni = load_node(xxx)

fleurinp_Fe = inpgen_calc_Fe.outputs.fleurinp
fleurinp_Co = inpgen_calc_Co.outputs.fleurinp
fleurinp_Ni = inpgen_calc_Ni.outputs.fleurinp

fleurinp_data_list = [fleurinp_Fe, fleurinp_Co, fleurinp_Ni]

As always, we need to initialise the other inputs:

In [ ]:
options = Dict(dict={'resources' : {"num_mpiprocs_per_machine" : 2}})

In [ ]:
wf_para = Dict(dict={'fleur_runmax' : 3,                    # passed to SCF workchain
                     'itmax_per_run' : 30,                  # passed to SCF workchain
                     'density_converged' : 0.002,           # passed to SCF workchain
                     'serial' : False,                      # passed to SCF workchain
                     'beta' : {'all' : 1.57079},            # sets beta angle for all atoms
                     'q_vectors': [[0.0, 0.0, 0.0,          # set q-vectors to calculate
                                   [0.125, 0.125, 0.0],     
                                   [0.250, 0.250, 0.0],     
                                   [0.375, 0.375, 0.0],     
                                   [0.500, 0.500, 0.0]],     
                     'ref_qss' : [0.0, 0.0, 0.0]            # sets a q-vector of the reference calc
                    })

In [ ]:
calc_parameters = Dict(dict={
    'kpt': {
        'div1': 12,
        'div2' : 10,
        'div3' : 1
        }})

In [ ]:
fleur_code = load_node(xxx)
inpgen_code = load_node(xxx)

In [ ]:
for fleuinp in fleurinp_data_list:
    workchain_pk = submit(FleurSSDispWorkChain,
                          fleur=fleur_code,
                          calc_parameters=calc_parameters,
                          wf_parameters=wf_para,
                          fleurinp=fleurinp)
    print('Submitted SSDisp workchain pk={} for {} structure'.format(workchain_pk, structure.get_formula()))